# Scrape products from amazon

In [120]:
pip show selenium beautifulsoup4 pandas

Name: selenium
Version: 4.25.0
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 
---
Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: soupsieve
Required-by: nbconvert
---
Name: pandas
Version: 2.2.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

Copyright (c) 2008-2011, AQR Capital Manag

In [121]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [128]:
# open chrome
driver = webdriver.Chrome()

In [129]:
# open the amazon page

driver.get('https://amazon.com/')

In [130]:
# open the driver page

driver.get('https://www.amazon.com/s?k=laptops&crid=2944PM084A0K4&sprefix=mobiles%2Caps%2C271&ref=nb_sb_noss_1')

In [131]:
# show html data of page

html_data = BeautifulSoup(driver.page_source, 'html.parser')

In [132]:
# print the html code of page
html_data

<html class="a-ws a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition a-ember" data-19ax5a9jf="dingo" data-aui-build-date="3.24.7-2024-09-17" lang="en-us"><!-- sp:feature:head-start --><head><script async="" crossorigin="anonymous" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics-incremental.min.js"></script><script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimiz

In [133]:
# get no of pages
No_of_pages = int(html_data.find('span',{'class':'s-pagination-item s-pagination-disabled'}).text)

In [134]:
# print no of pages
No_of_pages

20

In [135]:
products = html_data.find_all('div', {'data-component-type':'s-search-result'})

In [136]:
import time

In [137]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [138]:
from tqdm import tqdm

In [155]:
# extracting store data in arrays of product

titles = []
images = []
ratings = []
prices = []

for i in tqdm (range (No_of_pages)):
    url = 'https://www.amazon.com/s?k=laptops&crid=2944PM084A0K4&sprefix=mobiles%2Caps%2C271&ref=nb_sb_noss_1&page='+str(i+1)
    driver.get(url)
    time.sleep(3)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    products = html_data.find_all('div',{'data-component-type':'s-search-result'})
    
    for product in products:
        # Extract the title
        title = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}).text
        titles.append(title)
    
        # Extract the image source
        img = product.find('img')['src']
        images.append(img)
    
        # Extract the rating
        rating = product.find('span', {'class': 'a-icon-alt'})
        if rating is None:
            rating = 'No Rating'
        else:
            rating = rating.text
        ratings.append(rating)
    
        # Extract the price
        price_tag = product.find('span', {'class': 'a-price-whole'})
        if price_tag is None:
            price = 'No Price mentioned'
        else:
            price = '$' + price_tag.text
        prices.append(price)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:31<00:00,  7.57s/it]


In [149]:
# total products count
22 * 20

440

In [157]:
# print data of arrays and also validate the no of of products in lenght
print (titles)
print('---------------------------------------------------------------')
print()

len(titles)

['Acer Aspire Go 15 Slim Laptop | 15.6" Full HD IPS 1080P Display | Intel Core i3-N305| Intel UHD Graphics | 8GB LPDDR5 | 128GB HD | Wi-Fi 6 | AI PC | Windows 11 Home in S Mode | AG15-31P-3947', 'HP 17 Business Laptop, 17.3” HD+ Display, 11th Gen Intel Core i3-1125G4 Processor, 32GB RAM, 1TB SSD, Wi-Fi, HDMI, Webcam, Windows 11 Pro, Silver', 'HP 15.6" Portable Laptop (Include 1 Year Microsoft 365), HD Display, Intel Quad-Core N200 Processor, 16GB RAM, 128GB Storage, Wi-Fi 6, Webcam, HDMI, Numeric Keypad, Windows 11 Home, Black', 'HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 64 GB Storage, 14-inch Micro-edge HD Display, Windows 11 Home, Thin & Portable, 4K Graphics, One Year of Microsoft 365 (14-dq0040nr, Snowflake White)', 'Apple 2023 MacBook Pro Laptop M3 chip with 8‑core CPU, 10‑core GPU: Built for Apple Intelligence, 14.2-inch Liquid Retina XDR Display, 8GB Unified Memory, 512GB SSD Storage; Space Gray', 'Dell Inspiron 3511 Laptop, 15.6" Full HD Touchscreen, Intel Core i5-1135G7 (Be

329

In [158]:
# print data in table form to help to extract in excel file

data = pd.DataFrame({'titles':titles, 'images':images,'ratings':ratings, 'prices':prices})

In [159]:
# print the data
data

titles  \
0    Acer Aspire Go 15 Slim Laptop | 15.6" Full HD ...   
1    HP 17 Business Laptop, 17.3” HD+ Display, 11th...   
2    HP 15.6" Portable Laptop (Include 1 Year Micro...   
3    HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...   
4    Apple 2023 MacBook Pro Laptop M3 chip with 8‑c...   
..                                                 ...   
324  Dell Latitude 5310 Laptop - 13.3 FHD - 1.8 GHz...   
325  HP Envy x360 Convertible 13-bd0063dx Pale Gold...   
326  Dell Latitude 5530 Business Laptop, 15.6" FHD ...   
327  Lenovo ThinkPad X1 Carbon Gen 11 21HM000SUS 14...   
328  Dell Inspiron 13 5310, 13.3 inch QHD (Quad Hig...   

                                                images             ratings  \
0    https://m.media-amazon.com/images/I/71wKJVtV-+...  4.2 out of 5 stars   
1    https://m.media-amazon.com/images/I/61TGRBGTNz...  4.2 out of 5 stars   
2    https://m.media-amazon.com/images/I/71jnvo71a9...  4.2 out of 5 stars   
3    https://m.media-amazon.com/images/I/815uX7wkOZ...  3.9 out of 5 stars   
4    https://m.media-amazon.com/images/I/61lsexTCOh...  4.6 out of 5 stars   
..                                                 ...                 ...   
324  https://m.media-amazon.com/images/I/515cZvEK96...           No Rating   
325  https://m.media-amazon.com/images/I/31SNi7X7de...  4.3 out of 5 stars   
326  https://m.media-amazon.com/images/I/71a-a90XaD...  5.0 out of 5 stars   
327  https://m.media-amazon.com/images/I/61j0-o2+a-...  5.0 out of 5 stars   
328  https://m.media-amazon.com/images/I/81rTYNStzz...  4.1 out of 5 stars   

                 prices  
0    No Price mentioned  
1    No Price mentioned  
2    No Price mentioned  
3                 $181.  
4    No Price mentioned  
..                  ...  
324  No Price mentioned  
325               $559.  
326  No Price mentioned  
327  No Price mentioned  
328  No Price mentioned  

[329 rows x 4 columns]

In [160]:
# Make & load the csv file
data.to_csv('Laptops Products.csv')